In [1]:
# Read and unpack UMZ files
# https://docs.python.org/3/library/struct.html
import struct

infile = "umz/sandmark.umz"

program = []
with open(infile, mode='rb') as file:
    part = file.read(4)
    while part:
        program.append(struct.unpack('>L', part)[0])
        part = file.read(4)
        
print(len(program))
print(program[0])

14091
134217936


In [5]:
# decode standard instructions

p = program[0]     

op = p >> (32-4)
a = (p & 0b111000000) >> 6
b = (p & 0b000111000) >> 3
c = (p & 0b000000111) >> 0

print(op,a,b,c)

0 3 2 0


In [6]:
# decode orthography instruction (op == 13)

p = program[0]
for p in program:
    op = p >> (32-4)
    if op==13:
        break
        
a = p >> (32-4-3) & 0b111
value = p & 0b00000001111111111111111111111111           

print(op,a,value)

13 1 20


In [359]:
# console input
import sys
try:
    a = ord(input()) & 0xFF
except EOFError:
    a = 0xFFFFFFFF
print(a)

enfjrdjd


TypeError: ord() expected a character, but string of length 8 found

In [408]:
import struct

class UM:
    
    def __init__(self,infile=""):
        # platter arrays. 0 array initialized with program
        self.mem = [[]]
        if infile != "":
            self.readProg(infile)
        
        # registers
        self.reg = [0, 0, 0, 0, 0, 0, 0, 0]
            
        # instruction index ("finger")
        self.i = 0
        
        # input buffer
        self.command = []
        
        # status (0 = HALT, 1 = RUNNING, -1 = FAIL)
        self.status = 0
        
    def readProg(self,infile="umz/sandmark.umz"):
        with open(infile, mode='rb') as file:
            part = file.read(4)
            while part:
                self.mem[0].append(struct.unpack('>L', part)[0])
                part = file.read(4)

    ### OPERATORS
                
    def conditional_move(self,a,b,c):
        # 0. Conditional Move.
        # The register A receives the value in register B,
        # unless the register C contains 0.
        if self.reg[c] != 0: 
            self.reg[a] = self.reg[b]
        return

    def array_index(self,a,b,c):
        # 1. Array Index.
        # The register A receives the value stored at offset
        # in register C in the array identified by B.
        # ACTUALLY: identified by *value in register B!*
        self.reg[a] = self.mem[self.reg[b]][self.reg[c]]
        return

    def array_amendment(self,a,b,c):
        # 2. Array Amendment.
        # The array identified by A is amended at the offset
        # in register B to store the value in register C.
        # ACTUALLY: identified by *value in register A!*
        self.mem[self.reg[a]][self.reg[b]] = self.reg[c]
        return

    def addition(self,a,b,c):
        # 3. Addition.
        # The register A receives the value in register B plus 
        # the value in register C, modulo 2^32.
        self.reg[a] = (self.reg[b]+self.reg[c]) & 0b11111111111111111111111111111111
        return 

    def multiplication(self,a,b,c):
        # 4. Multiplication.
        # The register A receives the value in register B times
        # the value in register C, modulo 2^32.
        self.reg[a] = (self.reg[b]*self.reg[c]) & 0b11111111111111111111111111111111
        return

    def division(self,a,b,c):
        # 5. Division.
        # The register A receives the value in register B
        # divided by the value in register C, if any, where
        # each quantity is treated as an unsigned 32 bit number.
        if self.reg[c]>0:
            self.reg[a] = (self.reg[b] // self.reg[c])
        return

    def not_and(self,a,b,c):
        # 6. Not-And.
        # Each bit in the register A receives the 1 bit if
        # either register B or register C has a 0 bit in that
        # position.  Otherwise the bit in register A receives
        # the 0 bit.
        self.reg[a] = ( ~self.reg[b] & 0b11111111111111111111111111111111) | \
                       (~self.reg[c] & 0b11111111111111111111111111111111)
        return

    def halt(self,a,b,c):
        # 7. Halt.
        # The universal machine stops computation.
        self.status = 0
        return

    def allocation(self,a,b,c):
        # 8. Allocation.
        # A new array is created with a capacity of platters
        # commensurate to the value in the register C. This
        # new array is initialized entirely with platters
        # holding the value 0. A bit pattern not consisting of
        # exclusively the 0 bit, and that identifies no other
        # active allocated array, is placed in the B register.
        self.mem.append( [0] * self.reg[c] )
        self.reg[b] = len(self.mem)-1
        return

    def abandonement(self,a,b,c):
        # 9. Abandonment.
        # The array identified by the register C is abandoned.
        # Future allocations may then reuse that identifier.
        if self.reg[c]>0 and self.reg[c]<len(self.mem):
            self.mem[self.reg[c]] = []
        else:
            self.status = -1
            print(" ** Abandonment FAIL **")
        return      

    def output_(self,a,b,c):
        # 10. Output.
        # The value in the register C is displayed on the console
        # immediately. Only values between and including 0 and 255
        # are allowed.
        if self.reg[c]<=255:
            print(chr(self.reg[c]),end="")
        else:
            self.status = -1
            print(" ** Output FAIL **")
        return

    def input_(self,a,b,c):
        # 11. Input.
        # The universal machine waits for input on the console.
        # When input arrives, the register C is loaded with the
        # input, which must be between and including 0 and 255.
        # If the end of input has been signaled, then the 
        # register C is endowed with a uniform value pattern
        # where every place is pregnant with the 1 bit.
        if not len(self.command):
            self.command = list(input())
            self.i -= 1
            return
        else:
            while len(self.command):
                self.reg[c] = ord(self.command.pop(0)) & 0xFF # avoid values larger than 255
                return
            self.reg[c] = 0b11111111111111111111111111111111
            return

    def load_program(self,a,b,c):
        # 12. Load Program.
        # The array identified by the B register is duplicated
        # and the duplicate shall replace the '0' array,
        # regardless of size. The execution finger is placed
        # to indicate the platter of this array that is
        # described by the offset given in C, where the value
        # 0 denotes the first platter, 1 the second, et cetera.
        #
        # The '0' array shall be the most sublime choice for
        # loading, and shall be handled with the utmost
        # velocity.
        if self.reg[b] != 0: # do the copying only if needed!
            if len(self.mem[self.reg[b]]):
                self.mem[0] = list(self.mem[self.reg[b]])                
            else:
                self.status = -1
                print(" ** Load FAIL **")
        self.i = self.reg[c]-1 # subtract 1 since it'll be incremented later
        return
    
    def orthography(self,a,b,c):
        # 13. Orthography.
        # The value indicated is loaded into the register A
        # forthwith.
        self.reg[a] = b # replace v with b in call
    
    ### Operator map
    
    operator = [
        conditional_move,
        array_index,
        array_amendment,
        addition,
        multiplication,
        division,
        not_and,
        halt,
        allocation,
        abandonement,
        output_,
        input_,
        load_program,
        orthography,
    ]    
                
    def run(self):
        
        a = 0
        b = 0
        c = 0
        v = 0     
        self.status = 1
        
        while self.status>0:
            
            # If at the beginning of a cycle, the execution finger does not indicate 
            # a platter that describes a valid instruction, then the machine may Fail.   
            if self.i >= len(self.mem[0]):
                print(" ** Reached end of program **")
                return
            
            # get instruction from program array
            p = self.mem[0][self.i]
            
            # decode instruction
            op = p >> (32-4)
            if op==13:
                a = p >> (32-4-3) & 0b111
                v = p & 0b00000001111111111111111111111111
                self.operator[op](self,a,v,v)            
            else:
                a = (p & 0b111000000) >> 6
                b = (p & 0b000111000) >> 3
                c = (p & 0b000000111) >> 0
                self.operator[op](self,a,b,c)

            self.i += 1

In [ ]:
um = UM("umz/sandmark.umz")
um.run()

trying to Allocate array of size 0..
trying to Abandon size 0 allocation..
trying to Allocate size 11..
trying Array Index on allocated array..
trying Amendment of allocated array..
checking Amendment of allocated array..
trying Alloc(a,a) and amending it..
comparing multiple allocations..
pointer arithmetic..
check old allocation..
simple tests ok!
about to load program from some allocated array..
success.
verifying that the array and its copy are the same...
success.
testing aliasing..
success.
free after loadprog..
success.
loadprog ok.
